# Parsing Historical Prices of Brazilian Stocks

## Dependencies

In [ ]:
!pip install finbr

In [9]:
import pandas as pd
from finbr.b3 import cotahist
from datetime import datetime

## Processing

In [17]:
DATABASE_PATH = "prices.db"

stocks = [
    {"id": "ALUP11", "cnpj": "08.364.948/0001-38", "name": "Alupar Investimento"},
    {"id": "AURE3", "cnpj": "28.594.234/0001-23", "name": "Auren Energia"},
    {
        "id": "CPLE3",
        "cnpj": "76.483.817/0001-20",
        "name": "Companhia Paranaense de Energia",
    },
    {"id": "EGIE3", "cnpj": "02.474.103/0001-19", "name": "Engie Brasil Energia"},
    {"id": "ELET3", "cnpj": "00.001.180/0001-26", "name": "Eletrobrás"},
    {"id": "ENEV3", "cnpj": "04.423.567/0001-21", "name": "Eneva"},
    {"id": "ENGI3", "cnpj": "00.864.214/0001-06", "name": "Energisa"},
    {"id": "EQTL3", "cnpj": "03.220.438/0001-73", "name": "Equatorial"},
    {"id": "ISAE3", "cnpj": "02.998.611/0001-04", "name": "ISA Energia Brasil"},
    {"id": "LIGT3", "cnpj": "03.378.521/0001-75", "name": "Light"},
    {"id": "NEOE3", "cnpj": "01.083.200/0001-18", "name": "Neoenergia"},
    {"id": "RNEW11", "cnpj": "08.534.605/0001-74", "name": "Renova Energia"},
    {"id": "SRNA3", "cnpj": "42.500.384/0001-51", "name": "Serena Energia"},
]

stock_codes = [s["id"] for s in stocks]
stock2cnpj = {s["id"]: s["cnpj"] for s in stocks}

years = [2020, 2021, 2022, 2023, 2024, 2025]

In [11]:
dfs = []
for year in years:
    print(year)
    # baixa dados para um ano inteiro
    dados_anuais = cotahist.get_ano(year)
    dfs.append(dados_anuais[dados_anuais["CODIGO_DE_NEGOCIACAO"].isin(stock_codes)])

df = pd.concat(dfs)

2020
2021
2022
2023
2024
2025


In [13]:
df.columns

Index(['TIPO_DE_REGISTRO', 'DATA_DO_PREGAO', 'CODIGO_BDI',
       'CODIGO_DE_NEGOCIACAO', 'TIPO_DE_MERCADO', 'NOME_DA_EMPRESA',
       'ESPECIFICACAO_DO_PAPEL', 'PRAZO_EM_DIAS_DO_MERCADO_A_TERMO',
       'MOEDA_DE_REFERENCIA', 'PRECO_DE_ABERTURA', 'PRECO_MAXIMO',
       'PRECO_MINIMO', 'PRECO_MEDIO', 'PRECO_ULTIMO_NEGOCIO',
       'PRECO_MELHOR_OFERTA_DE_COMPRA', 'PRECO_MELHOR_OFERTA_DE_VENDAS',
       'NUMERO_DE_NEGOCIOS', 'QUANTIDADE_NEGOCIADA', 'VOLUME_TOTAL_NEGOCIADO',
       'PRECO_DE_EXERCICIO', 'INDICADOR_DE_CORRECAO_DE_PRECOS',
       'DATA_DE_VENCIMENTO', 'FATOR_DE_COTACAO',
       'PRECO_DE_EXERCICIO_EM_PONTOS', 'CODIGO_ISIN',
       'NUMERO_DE_DISTRIBUICAO'],
      dtype='object')

In [14]:
import sqlite3

command = """CREATE TABLE IF NOT EXISTS COTAHIST (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
	CNPJ TEXT NOT NULL,
	DATA_DO_PREGAO DATETIME NOT NULL,
	NOME_DA_EMPRESA TEXT NOT NULL,
	CODIGO_DE_NEGOCIACAO TEXT NOT NULL,
	ESPECIFICACAO_DO_PAPEL TEXT NOT NULL,
	MOEDA_DE_REFERENCIA TEXT NOT NULL,
	PRECO_DE_ABERTURA FLOAT NOT NULL,
 	PRECO_MAXIMO FLOAT NOT NULL,
	PRECO_MINIMO FLOAT NOT NULL,
 	PRECO_MEDIO FLOAT NOT NULL,
	PRECO_ULTIMO_NEGOCIO FLOAT NOT NULL,
	PRECO_MELHOR_OFERTA_DE_COMPRA FLOAT NOT NULL,
 	NUMERO_DE_NEGOCIOS INT NOT NULL,
	QUANTIDADE_NEGOCIADA INT NOT NULL,
	VOLUME_TOTAL_NEGOCIADO INT NOT NULL
);

-- Índice para consultas por CNPJ
CREATE INDEX IF NOT EXISTS idx_cotahist_cnpj 
ON COTAHIST (CNPJ);

-- Índice para consultas por data de início de período
CREATE INDEX IF NOT EXISTS idx_cotahist_pregao_date 
ON COTAHIST (DATA_DO_PREGAO);"""

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(DATABASE_PATH)

# Create a cursor object using the connection
cursor = connection.cursor()

# Execute the command
cursor.executescript(command)

# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

In [18]:
queries = []


fields = [
    "CNPJ",
    "DATA_DO_PREGAO",
    "NOME_DA_EMPRESA",
    "CODIGO_DE_NEGOCIACAO",
    "ESPECIFICACAO_DO_PAPEL",
    "MOEDA_DE_REFERENCIA",
    "PRECO_DE_ABERTURA",
    "PRECO_MAXIMO",
    "PRECO_MINIMO",
    "PRECO_MEDIO",
    "PRECO_ULTIMO_NEGOCIO",
    "PRECO_MELHOR_OFERTA_DE_COMPRA",
    "NUMERO_DE_NEGOCIOS",
    "QUANTIDADE_NEGOCIADA",
    "VOLUME_TOTAL_NEGOCIADO",
]
fields_str = ", ".join(fields)
placeholders = ", ".join(["?"] * len(fields))

query = f"INSERT INTO COTAHIST ({fields_str}) VALUES ({placeholders})"

data = []  # list of tuples to insert

for idx, row in df.iterrows():
    row_values = []
    for field in fields:
        value = (
            row[field] if field != "CNPJ" else stock2cnpj[row["CODIGO_DE_NEGOCIACAO"]]
        )
        if isinstance(value, datetime):
            try:
                value = value.strftime("%Y-%m-%d")
            except Exception:
                value = "NULL"
        elif isinstance(value, float) and (value != value):  # NaN check
            value = None
        elif value == "NULL" or value == "":
            value = None
        row_values.append(value)

    data.append(tuple(row_values))

# Connect to a SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect(DATABASE_PATH)

# Create a cursor object using the connection
cursor = connection.cursor()

# Execute the command
# Bulk insert safely
cursor.executemany(query, data)
# Commit the changes to the database
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()